# Method 6

In [8]:
import numpy as np
import pandas as pd
from scipy.cluster.vq import kmeans,vq
import shutil
import os
import numpy as np
import pandas as pd
import shutil
import os
import glob, os, errno
import sys
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
import cv2
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
from os import listdir
from matplotlib import image
from natsort import natsorted
import pandas as pd 
from numpy import array
from PIL import Image
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
import os.path
from sklearn.metrics import accuracy_score
from os import path
import sys
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.ensemble import BaggingClassifier
import cv2
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
from os import listdir
from matplotlib import image
from natsort import natsorted
import pandas as pd 

In [12]:
def readImages(direc,dest): # reading images with denosing the imges
    for filename in natsorted(listdir(direc)):
        image = cv2.imread(direc+filename)
        dst = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)
        cv2.imwrite(dest+filename,image)

In [13]:
readImages('./bonus-sml-2020/SML_Train/','./bonus-sml-2020/SML_Train_Temp/')

In [15]:
train_files=natsorted(listdir('./bonus-sml-2020/SML_Train_Temp/')) # split train data into traain and validation
train_labels = pd.read_csv("./bonus-sml-2020/SML_Train.csv")['category'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(train_files, train_labels, test_size=0.2)
#X_train,X_test has image file names 

def splitting(X_test,direc,dest):
    for filename in X_test:
        image = cv2.imread(direc+filename)
        cv2.imwrite(dest+filename,image)
        os.remove(direc+filename)
        
splitting(X_test,'./bonus-sml-2020/SML_Train_Temp/','./bonus-sml-2020/SML_Validation_new/')

In [16]:
labels = pd.read_csv("./bonus-sml-2020/SML_Train.csv") #dividing into folders for train data based on class

# Create `train_sep` directory
train_dir = './bonus-sml-2020/SML_Train_temp/'
train_sep_dir = './bonus-sml-2020/train_sep_new/'
if not os.path.exists(train_sep_dir):
    os.mkdir(train_sep_dir)

for filename,class_name in labels.values:
    if path.exists((train_dir + filename)):
        # Create subdirectory with `class_name`
        if not os.path.exists(train_sep_dir + str(class_name)):
            os.mkdir(train_sep_dir + str(class_name))
        src_path = train_dir + filename
        dst_path = train_sep_dir + str(class_name) + '/' + filename
        try:
            shutil.copy(src_path, dst_path)
        except IOError as e:
            print('Unable to copy file {} to {}'
                  .format(src_path, dst_path))
        except:
            print('When try copy file {} to {}, unexpected error: {}'
                  .format(src_path, dst_path, sys.exc_info()))
            
# all bove code were input

In [17]:
train_path='./bonus-sml-2020/train_sep_new/'
training_names=os.listdir(train_path)
image_paths=[]
image_classes=[]

def imglist(path):
    return[os.path.join(path,f) for f in os.listdir(path)]


for training_name in training_names:
    dir=os.path.join(train_path,training_name)
    class_path=imglist(dir)
    image_paths+=class_path #paths of all images
    image_classes+=[training_name]*len(class_path) #class of all images
# print(image_paths[9000])
# print(image_classes[9000])
    
des_list=[]  # using to find the descriptors of the images
brisk=cv2.BRISK_create(30)

count=0
for image_path in image_paths:
    im=cv2.imread(image_path)
    kpts,des=brisk.detectAndCompute(im,None)
    if des is None:
        count+=1
        no_kp = np.zeros((1, brisk.descriptorSize()), np.float32)
        des_list.append((image_path,no_kp))
    else:   
        des_list.append((image_path,des))
print(count)#none decriptors

170


In [18]:
descriptors=des_list[0][1]
for image_path,descriptor in des_list[1:]:
    descriptors=np.vstack((descriptors,descriptor))

In [19]:
descriptors_float=descriptors.astype(float) # using KNN modle to forem cluster
k=50
voc,variance=kmeans(descriptors_float,k,1)

In [20]:
im_features=np.zeros((len(des_list),k),"float32")
for i in range(len(des_list)):
    words,distance=vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w]+=1

In [ ]:
# clf = RandomForestClassifier(n_estimators=100, random_state=30)
# clf.fit(im_features, np.array(image_classes))

In [ ]:
# clf=SVC(C=5, cache_size=200,class_weight='balanced', coef0=0.0,decision_function_shape='ovr',degree=3, gamma=0.005, kernel='rbf',
#         max_iter=-1, probability=False,random_state=None, shrinking=True,tol=0.001, verbose=False)
# clf.fit(im_features,np.array(image_classes))

In [21]:
labels = pd.read_csv("./bonus-sml-2020/SML_Train.csv") #dividing into folders for train data

# Create `train_sep` directory
train_dir = './bonus-sml-2020/SML_Validation_new/'
train_sep_dir = './bonus-sml-2020/validation_sep_new/'
if not os.path.exists(train_sep_dir):
    os.mkdir(train_sep_dir)

for filename,class_name in labels.values:
    if path.exists((train_dir + filename)):
        # Create subdirectory with `class_name`
        if not os.path.exists(train_sep_dir + str(class_name)):
            os.mkdir(train_sep_dir + str(class_name))
        src_path = train_dir + filename
        dst_path = train_sep_dir + str(class_name) + '/' + filename
        try:
            shutil.copy(src_path, dst_path)
        except IOError as e:
            print('Unable to copy file {} to {}'
                  .format(src_path, dst_path))
        except:
            print('When try copy file {} to {}, unexpected error: {}'
                  .format(src_path, dst_path, sys.exc_info()))
            
# all bove code were input

In [22]:
validation_path='./bonus-sml-2020/validation_sep_new/' # doing all above task for validation data as well finding teh dexriports
validation_names=os.listdir(validation_path)
validation_image_paths=[]
validation_image_classes=[]

def imglist(path):
    return[os.path.join(path,f) for f in os.listdir(path)]


for validation_name in validation_names:
    dir=os.path.join(validation_path,validation_name)
    class_path=imglist(dir)
    validation_image_paths+=class_path #paths of all images
    validation_image_classes+=[validation_name]*len(class_path) #class of all images
# print(image_paths[9000])
# print(image_classes[9000])
    
validation_des_list=[]
brisk=cv2.BRISK_create(30)

count=0
for validation_image_path in validation_image_paths:
    im=cv2.imread(validation_image_path)
    kpts,des=brisk.detectAndCompute(im,None)
    if des is None:
        count+=1
        no_kp = np.zeros((1, brisk.descriptorSize()), np.float32)
        validation_des_list.append((validation_image_path,no_kp))
    else:   
        validation_des_list.append((validation_image_path,des))
print(count)#none decriptors

35


In [23]:
validation_descriptors=validation_des_list[0][1]
for validation_image_path,validation_descriptor in validation_des_list[1:]:
    validation_descriptors=np.vstack((validation_descriptors,validation_descriptor))

In [24]:
validation_im_features=np.zeros((len(validation_des_list),k),"float32")
for i in range(len(validation_des_list)):
    words,distance=vq(validation_des_list[i][1],voc)
    for w in words:
        validation_im_features[i][w]+=1

In [14]:
#model 1
pca = PCA(n_components=150, whiten=True, random_state=42,svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(im_features,np.array(image_classes))
model = grid.best_estimator_
validation_y_pred = model.predict(validation_im_features)
#validation_y_pred = clf.predict(validation_im_features)
accuracy_score(validation_y_pred,np.array(validation_image_classes))

NameError: name 'im_features' is not defined

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [25]:
#model 2
clf=SVC(C=5, cache_size=200,class_weight='balanced', coef0=0.0,decision_function_shape='ovr',degree=3, gamma=0.005, kernel='rbf',
         max_iter=-1, probability=False,random_state=None, shrinking=True,tol=0.001, verbose=False)

%time clf.fit(im_features,np.array(image_classes))
validation_y_pred = clf.predict(validation_im_features)
#validation_y_pred = clf.predict(validation_im_features)
accuracy_score(validation_y_pred,np.array(validation_image_classes))

CPU times: user 20.5 s, sys: 166 ms, total: 20.7 s
Wall time: 21 s


0.1625

In [ ]:
#model 3
clf = RandomForestClassifier(n_estimators=100, random_state=30)
%time clf.fit(im_features,np.array(image_classes))
validation_y_pred = model.predict(validation_im_features)
#validation_y_pred = clf.predict(validation_im_features)
accuracy_score(validation_y_pred,np.array(validation_image_classes))

In [ ]:
#model 4
clf = BaggingClassifier(base_estimator=svm, n_estimators=31, random_state=42)
%time clf.fit(im_features,np.array(image_classes))
validation_y_pred = model.predict(validation_im_features)
#validation_y_pred = clf.predict(validation_im_features)
accuracy_score(validation_y_pred,np.array(validation_image_classes))

In [30]:
readImages('./bonus-sml-2020/SML_Test/','./bonus-sml-2020/SML_Test_Temp/') #contains clean images

In [31]:
test_path='./bonus-sml-2020/SML_Test_Temp/'# finding decrpitors of etst data
test_names=os.listdir(test_path)
test_image_paths=[]

for test_name in test_names:
    test_image_paths.append(test_path+test_name) #paths of all images
print(test_image_paths[0])
test_des_list=[]
brisk=cv2.BRISK_create(30)

count=0
for test_image_path in test_image_paths:
    im=cv2.imread(test_image_path)
    kpts,des=brisk.detectAndCompute(im,None)
    if des is None:
        count+=1
        no_kp = np.zeros((1, brisk.descriptorSize()), np.float32)
        test_des_list.append((test_image_path,no_kp))
    else:   
        test_des_list.append((image_path,des))
print(count)#none decriptors

./bonus-sml-2020/SML_Test_Temp/Test_588.jpg
12


In [32]:
test_descriptors=test_des_list[0][1]
for test_image_path,test_descriptor in test_des_list[1:]:
    test_descriptors=np.vstack((test_descriptors,test_descriptor))

In [33]:
test_im_features=np.zeros((len(test_des_list),k),"float32")
for i in range(len(test_des_list)):
    words,distance=vq(test_des_list[i][1],voc)
    for w in words:
        test_im_features[i][w]+=1

In [35]:
y_pred = clf.predict(test_im_features)
#y_pred = clf.predict(test_im_features)

In [36]:
def helper(direc):
    files=[]
    for filename in natsorted(listdir(direc)):
        files.append(filename)
    return files

files=helper('./bonus-sml-2020/SML_Test/')
data_res=pd.DataFrame()
data_res['id']=files
data_res['category']=y_pred
data_res.to_csv (r'sample_submission.csv', index = False, header=True)

In [38]:
# dump model 
import joblib
filename = './bonus-sml-2020/MT19124_model/Other_Method_3/Other_Method_3.sav'

joblib.dump(clf, filename)

['./bonus-sml-2020/MT19124_model/Other_Method_3/Other_Method_3.sav']